# Step-by-step run of alphazero self-play & training.


In [ ]:
import os
import time
from pathlib import Path
from collections import defaultdict, Counter
import asyncio
from typing import Callable

import numpy as np
import torch
import torch.nn.functional as F

# Game and players
from rgi.rgizero.games.connect4 import Connect4Game
from rgi.rgizero.players.alphazero import AlphazeroPlayer
from rgi.rgizero.players.alphazero import play_game

from notebook_utils import reload_local_modules

print("✅ Imports successful")

if torch.cuda.is_available():
    device = 'cuda'
elif torch.backends.mps.is_available():
  device = 'mps'
else:
    device = 'cpu'
print(f'Using device: {device}')
assert device in ('cuda', 'mps'), f"No accelerator available, device={device}"

# Allow asyncio to work with jupyter notebook
import nest_asyncio
nest_asyncio.apply()

# Increase numpy print width
np.set_printoptions(linewidth=300)

%load_ext line_profiler

In [ ]:
DEBUG_MODE = True     # Set options to make debugger work properly. Single worker, etc.
LOAD_MODEL = False
TRAIN_MODEL = True
MODEL_SIZE = "tiny"  # "tiny" or "small" or "large" or "xl"
NUM_SIMULATIONS = 200
RUN_GENERATIONS = True
RUN_TOURNAMENT = False

# If False, we still load previous games from disk.
NUM_GAMES = 10_000
MAX_TRAINING_EPOCHS = 10
TRAIN_BATCH_SIZE = 2048
MAX_TRAINING_ITERS = 1_000_000 // TRAIN_BATCH_SIZE
CONFIG_ALIAS = f'trajectory_sims-{NUM_SIMULATIONS}_games-{NUM_GAMES}_size-{MODEL_SIZE}_train-{MAX_TRAINING_ITERS}_x1'
NUM_GENERATIONS = 20

# DEBUG: Update batch_size after config_alias
MODEL_SIZE = "small"
MAX_TRAINING_ITERS = 100_000_000 // TRAIN_BATCH_SIZE
MAX_TRAINING_EPOCHS = 10_000

## Step 1: Set up history-wrapped game


In [ ]:
from rgi.rgizero.games.history_wrapper import HistoryTrackingGame
from rgi.rgizero.data.trajectory_dataset import Vocab
from rgi.rgizero.common import TOKENS

base_game, max_game_length = Connect4Game(connect_length=4), 7*6

game = HistoryTrackingGame(base_game)
state_0 = game.initial_state()
block_size = max_game_length + 2
all_actions = game.all_actions()
action_vocab = Vocab(itos=[TOKENS.START_OF_GAME] + list(all_actions))
n_max_context = max_game_length + 2
game_name = base_game.__class__.__name__

print("✅ Using HistoryTrackingGame from module")
print(f"Game: {game_name}, Players: {game.num_players(state_0)}, Actions: {list(game.all_actions())}")

DATA_DIR = Path.cwd().parent / "data" / "rgizero-e2e" / game_name / CONFIG_ALIAS
print("Creating data dir: ", DATA_DIR)
os.makedirs(DATA_DIR, exist_ok=True)

MODEL_DIR = Path.cwd().parent / "models" / "rgizero-e2e" / game_name / CONFIG_ALIAS
print("Creating model dir: ", MODEL_DIR)
os.makedirs(MODEL_DIR, exist_ok=True)


## Step 2: Create random generation_0 model


In [ ]:
def print_dataset_stats(dataset_path, split_name):
    """Print statistics about a loaded trajectory dataset."""
    td = TrajectoryDataset(dataset_path.parent, split_name, block_size=n_max_context)
    
    # Calculate basic stats
    num_trajectories = len(td)
    total_actions = td._num_actions
    avg_trajectory_length = total_actions / num_trajectories if num_trajectories > 0 else 0
    
    # Get trajectory lengths, winners, and first moves
    trajectory_lengths = []
    winners = []
    first_moves = []
    
    for i in range(num_trajectories):
        start_idx = td.boundaries[i]
        end_idx = td.boundaries[i + 1]
        traj_length = end_idx - start_idx
        trajectory_lengths.append(traj_length)
        
        # Get winner from final values (values are the same throughout trajectory)
        # Values are in range [-1, 1] where positive means player 1 advantage
        final_values = td.value_data[start_idx]  # shape: (num_players,)
        if final_values[0] > final_values[1]:
            winners.append(1)
        elif final_values[1] > final_values[0]:
            winners.append(2)
        else:
            winners.append(None)  # Draw
        
        # Get first move (decode from vocab)
        first_action_encoded = td.action_data[start_idx]
        first_action = action_vocab.decode([first_action_encoded])[0]
        first_moves.append(first_action)
    
    # Print basic stats
    print(f"Dataset Stats:")
    print(f"  Trajectories: {num_trajectories}")
    print(f"  Total actions: {total_actions}")
    print(f"  Avg trajectory length: {avg_trajectory_length:.2f}")
    print(f"  Trajectory length - min: {min(trajectory_lengths)}, max: {max(trajectory_lengths)}, mean: {np.mean(trajectory_lengths):.2f}")
    
    # Print winner stats (similar to print_game_stats)
    print(f"Winner Stats:")
    winner_stats = Counter(winners)
    total_games = num_trajectories
    win1_pct = 100 * winner_stats[1] / total_games if total_games > 0 else 0
    win2_pct = 100 * winner_stats[2] / total_games if total_games > 0 else 0
    print(f"  Winner counts: win[1]={win1_pct:.2f}% win[2]={win2_pct:.2f}%, n={total_games}")
    
    # Print stats by initial move
    print(f"Winner Stats by initial move:")
    move_stats = defaultdict(Counter)
    for first_move, winner in zip(first_moves, winners):
        move_stats[first_move][winner] += 1
    
    for action in sorted(move_stats.keys()):
        counts = move_stats[action]
        total = sum(counts.values())
        win1_pct = 100 * counts[1] / total if total > 0 else 0
        win2_pct = 100 * counts[2] / total if total > 0 else 0
        draw_pct = 100 * counts[None] / total if total > 0 else 0
        print(f"  a={action}: n={total:3} win[1]={win1_pct:.2f}% counts={counts}, win[2]={win2_pct:.2f}% draw={draw_pct:.2f}%")

def print_model_stats(model, config_alias=""):
    """Print statistics about a model."""
    # Count parameters
    num_params = sum(p.numel() for p in model.parameters())
    num_trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
    
    print(f"Model Stats:")
    print(f"  Config: {model.config}")
    print(f"  Total parameters: {num_params:,}")
    print(f"  Trainable parameters: {num_trainable:,}")
    if config_alias:
        print(f"  Config alias: {config_alias}")


In [ ]:
reload_local_modules(verbose=False)

from rgi.rgizero.models.action_history_transformer import ActionHistoryTransformer
from rgi.rgizero.models.transformer import TransformerConfig

model_config_dict = {
    "tiny": TransformerConfig(n_max_context=n_max_context, n_layer=2, n_head=2, n_embd=8),
    "small": TransformerConfig(n_max_context=n_max_context, n_layer=4, n_head=4, n_embd=32),
    "large": TransformerConfig(n_max_context=n_max_context, n_layer=8, n_head=8, n_embd=128),
    "xl": TransformerConfig(n_max_context=n_max_context, n_layer=16, n_head=16, n_embd=256),
}

from rgi.rgizero.models.tuner import create_random_model

# Make model initialization deterministic
model_config = model_config_dict[MODEL_SIZE]
# TODO: Use MODEL_SIZE!
# model_config = model_config_dict["small"] # Override to see if we can fit better.
model_0 = create_random_model(model_config, action_vocab_size=action_vocab.vocab_size, num_players=game.num_players(state_0), seed=42, device=device)

# Step 3: Define play & generation code


In [ ]:
from rgi.rgizero.models.action_history_transformer import AsyncNetworkEvaluator, ActionHistoryTransformerEvaluator
from rgi.rgizero.players.alphazero import play_game_async
from tqdm.asyncio import tqdm

async def play_games_async(num_games: int, player_factory: Callable[[], AlphazeroPlayer], max_concurrent_games: int = 1000):
    sem = asyncio.Semaphore(max_concurrent_games)
    tasks = []
    async def create_player_and_create_game():
        async with sem:
            t0 = time.time()
            player = player_factory()
            game_result = await play_game_async(game, [player, player])
            t1 = time.time()
            game_result['time'] = t1 - t0
            return game_result

    tasks = [create_player_and_create_game() for _ in range(num_games)]
    results = await tqdm.gather(*tasks)   # same as asyncio.gather, but with a progress bar
    return results

async def play_generation_async(model, num_games, simulations=NUM_SIMULATIONS, max_concurrent_games=1024):
    serial_evaluator = ActionHistoryTransformerEvaluator(model, device=device, block_size=block_size, vocab=action_vocab)
    async_evaluator = AsyncNetworkEvaluator(base_evaluator=serial_evaluator, max_batch_size=max_concurrent_games, verbose=False)

    master_rng = np.random.default_rng(42)
    async_player_factory = lambda: AlphazeroPlayer(game, async_evaluator, rng=np.random.default_rng(master_rng.integers(0, 2**31)), add_noise=False, simulations=simulations)

    await async_evaluator.start()
    results = await play_games_async(num_games=num_games, player_factory=async_player_factory, max_concurrent_games=max_concurrent_games)
    await async_evaluator.stop()
    return results

def print_game_stats(results):
    print("Winner Stats:")
    winner_stats = Counter(result['winner'] for result in results)
    print(f"Winner counts: win[1]={100*winner_stats[1]/sum(winner_stats.values()):.2f}% win[2]={100*winner_stats[2]/sum(winner_stats.values()):.2f}%, n={sum(winner_stats.values())}, draw={100*winner_stats[None]/sum(winner_stats.values()):.2f}%")
    game_lengths = [len(result['action_history']) for result in results]
    print(f"Game Length min: {min(game_lengths)}, max: {max(game_lengths)}, mean: {np.mean(game_lengths):.2f}")
    print("Winner Stats by initial move:")
    dd = defaultdict(Counter)
    for result in results:
        dd[result['action_history'][0]][result['winner']] += 1
    for action, counts in sorted(dd.items()):
        print(f"  a={action}: n={sum(counts.values()):3} win[1]={100*counts[1]/sum(counts.values()):.2f}% counts={counts}, win[2]={100*counts[2]/sum(counts.values()):.2f}% draw={100*counts[None]/sum(counts.values()):.2f}%")

## Step 3: Confirm we can read & write to trajectory_dataset


In [ ]:
from rgi.rgizero.data.trajectory_dataset import TrajectoryDatasetBuilder, TrajectoryDataset
reload_local_modules(verbose=False)

def add_trajectory(game_result, vocab, td_builder):
    action_history = game_result['action_history']
    trajectory_length = len(action_history)
    legal_policies = game_result['legal_policies']
    legal_action_idx = game_result['legal_action_idx']
    rewards = game_result['rewards']

    # Translation key for converting legal_action_ids to vocab_action_idx.
    action_idx_to_vocab_idx = vocab.encode(all_actions)

    fixed_width_policies = np.zeros((trajectory_length, vocab.vocab_size))
    for i in range(trajectory_length):
        vocab_action_idx = action_idx_to_vocab_idx[legal_action_idx[i]]
        fixed_width_policies[i, vocab_action_idx] = legal_policies[i]

    encoded_action_history = vocab.encode(action_history)
    tiled_rewards = np.tile(rewards, (trajectory_length, 1))  # shape (num_players,) -> (num_moves, num_players)
    
    td_builder.add_trajectory(actions=encoded_action_history, fixed_width_policies=fixed_width_policies, values=tiled_rewards)

def write_trajectory_dataset(results, action_vocab, generation_id):
    td_builder = TrajectoryDatasetBuilder(action_vocab)
    for game_result in results:
        add_trajectory(game_result, action_vocab, td_builder)

    trajectory_path = td_builder.save(DATA_DIR, f"gen-{generation_id}")
    return trajectory_path

## Train model


In [ ]:
from rgi.rgizero.train import Trainer, TrainConfig

LEARNING_RATE = 0.05

train_config = TrainConfig(
    model_name="connect4-e2e",
    model_version="v1",

    eval_interval = 1000,  # keep frequent because we'll overfit
    eval_iters = 20,
    log_interval = 100,  # don't print too too often
    max_epochs = MAX_TRAINING_EPOCHS,

    # we expect to overfit on this small dataset, so only save when val improves
    always_save_checkpoint = False,

    gradient_accumulation_steps = 1,
    batch_size = TRAIN_BATCH_SIZE,

    learning_rate = LEARNING_RATE,  # with baby networks can afford to go a bit higher
    max_iters = MAX_TRAINING_ITERS,
    lr_decay_iters = MAX_TRAINING_ITERS,  # make equal to max_iters usually
    min_lr = LEARNING_RATE / 10,  # learning_rate / 10 usually
    beta2 = 0.99,  # make a bit bigger because number of tokens per iter is small

    warmup_iters = 0,  # not super necessary potentially
)

num_workers = 0 if DEBUG_MODE else 4

def train_model(model, training_splits, train_config):
    from rgi.rgizero.models.tuner import train_model as tuner_train_model
    model, trainer = tuner_train_model(model, training_splits, train_config, device, n_max_context, DATA_DIR, num_workers)
    return model, trainer

In [ ]:
import dataclasses

def get_model_path(generation_id):
    return MODEL_DIR / f"gen-{generation_id}.pt"

def save_model(model, trainer, generation_id):
    # Save model
    model_path = get_model_path(generation_id)

    checkpoint = {
        'model': model.state_dict(),
        'model_config': dataclasses.asdict(model.config),
        'vocab': action_vocab.to_dict(),
        'iter_num': trainer.iter_num,
        'best_val_loss': trainer.best_val_loss,
        'num_players': game.num_players(state_0),
    }
    torch.save(checkpoint, model_path)
    return model_path

def load_model(generation_id):
    model_path = get_model_path(generation_id)
    loaded_checkpoint = torch.load(model_path)
    loaded_model = ActionHistoryTransformer(
        config=TransformerConfig(**loaded_checkpoint['model_config']),
        action_vocab_size=loaded_checkpoint['vocab']['vocab_size'],
        num_players=loaded_checkpoint['num_players']
    )
    loaded_model.load_state_dict(loaded_checkpoint['model']) 
    loaded_model.to(device)
    return loaded_model


In [ ]:
# Do a single generation of play & train
async def run_generation(model, num_games, num_simulations, generation_id):
    print(f"\n\n## Running generation {generation_id} for config_alias={CONFIG_ALIAS}")
    split_name = f"gen-{generation_id}"
    expected_trajectory_path = DATA_DIR / split_name
    if not expected_trajectory_path.exists():
        print(f"Playing {num_games} games, simulations={num_simulations}, model_size={MODEL_SIZE}")
        results = await play_generation_async(model, num_games=NUM_GAMES, simulations=NUM_SIMULATIONS)
        print_game_stats(results)
        trajectory_path = write_trajectory_dataset(results, action_vocab, generation_id)
        assert trajectory_path == expected_trajectory_path
    else:
        print(f"Loading trajectory from {expected_trajectory_path}")
        print_dataset_stats(expected_trajectory_path, split_name)
        trajectory_path = expected_trajectory_path
        results = None

    model_path = get_model_path(generation_id)
    if not model_path.exists():
        print(f"Training model on {split_name}")
        training_splits = [f"gen-{i}" for i in range(1, generation_id+1)]
        # TODO: We're continuing training on a previosu model here ... should we train a new model from scratch?
        print(train_config)
        updated_model, trainer = train_model(model, training_splits, train_config)
        save_model(updated_model, trainer, generation_id)
    else:
        print(f"Loading model from {model_path}")
        updated_model = load_model(generation_id)
        print_model_stats(updated_model, config_alias=MODEL_SIZE)

    return results, trajectory_path, updated_model

In [ ]:
results_dict = {}
trajectory_paths_dict = {}
model_dict = {0: model_0}

current_model = model_dict[0]
if RUN_GENERATIONS:
    for generation_id in range(1, NUM_GENERATIONS+1):
        current_model = model_dict[generation_id-1]
        results_i, trajectory_path_i, model_i = await run_generation(current_model, num_games=NUM_GAMES, num_simulations=NUM_SIMULATIONS, generation_id=generation_id)
        results_dict[generation_id] = results_i
        trajectory_paths_dict[generation_id] = trajectory_path_i
        model_dict[generation_id] = model_i

## refactor, learning_rate = 0.05, warmup_iters=0
# step 0: train loss 2.7801, val loss 2.7801
# iter 0/5/488: loss 2.7801, time 611.02ms
# iter 100/105/488: loss 2.5840, time 63.73ms
# iter 200/205/488: loss 2.5958, time 62.98ms
# iter 300/305/488: loss 2.5835, time 60.15ms
# iter 400/405/488: loss 2.5793, time 63.62ms

# ## model = small
# step 0: train loss 2.7741, val loss 2.7741
# iter 0/5/488: loss 2.7743, time 1624.89ms
# iter 100/105/488: loss 2.6157, time 141.39ms
# iter 200/205/488: loss 2.6120, time 161.22ms
# iter 300/305/488: loss 2.5983, time 203.82ms


# using fused AdamW: False
# step 0: train loss 2.7801, val loss 2.7801
# iter 0/5/4882: loss 2.7801, time 1422.53ms
# iter 100/105/4882: loss 2.5970, time 110.71ms
# iter 200/205/4882: loss 2.5962, time 116.96ms
# iter 300/305/4882: loss 2.5917, time 160.95ms
# iter 400/405/4882: loss 2.5885, time 63.37ms
# iter 500/505/4882: loss 2.5912, time 65.25ms
# iter 600/605/4882: loss 2.6000, time 67.49ms
# iter 700/705/4882: loss 2.5780, time 61.43ms
# iter 800/805/4882: loss 2.5864, time 265.56ms
# iter 900/905/4882: loss 2.5857, time 263.09ms
# step 1000: train loss 2.5849, val loss 2.5847
# saving checkpoint to /Users/rodo/src/rgi3/models/connect4-e2e/v1
# iter 1000/1005/4882: loss 2.5844, time 1812.58ms
# iter 1100/1105/4882: loss 2.5832, time 62.89ms
# iter 1200/1205/4882: loss 2.5743, time 95.25ms
# iter 1300/1305/4882: loss 2.5720, time 324.18ms
# iter 1400/1405/4882: loss 2.5880, time 73.66ms
# iter 1500/1505/4882: loss 2.5745, time 295.39ms
# iter 1600/1605/4882: loss 2.5726, time 76.05ms
# iter 1700/1705/4882: loss 2.5670, time 63.20ms
# iter 1800/1805/4882: loss 2.5720, time 62.66ms
# iter 1900/1905/4882: loss 2.5694, time 449.06ms
# step 2000: train loss 2.5806, val loss 2.5806
# saving checkpoint to /Users/rodo/src/rgi3/models/connect4-e2e/v1
# iter 2000/2005/4882: loss 2.5893, time 920.12ms
# iter 2100/2105/4882: loss 2.5686, time 430.53ms
# iter 2200/2205/4882: loss 2.5741, time 63.05ms
# iter 2300/2305/4882: loss 2.5679, time 60.90ms
# iter 2400/2405/4882: loss 2.5754, time 69.07ms
# iter 2500/2505/4882: loss 2.5673, time 68.33ms
# iter 2600/2605/4882: loss 2.5648, time 66.26ms
# iter 2700/2705/4882: loss 2.5622, time 69.76ms
# iter 2800/2805/4882: loss 2.5541, time 143.65ms
# iter 2900/2905/4882: loss 2.5634, time 66.40ms
# step 3000: train loss 2.5550, val loss 2.5547
# saving checkpoint to /Users/rodo/src/rgi3/models/connect4-e2e/v1
# iter 3000/3005/4882: loss 2.5545, time 975.15ms
# iter 3100/3105/4882: loss 2.5594, time 63.55ms
# iter 3200/3205/4882: loss 2.5499, time 64.17ms
# iter 3300/3305/4882: loss 2.5481, time 70.28ms
# iter 3400/3405/4882: loss 2.5565, time 73.58ms
# iter 3500/3505/4882: loss 2.5602, time 72.22ms
# iter 3600/3605/4882: loss 2.5429, time 88.68ms
# iter 3700/3705/4882: loss 2.5259, time 63.15ms
# iter 3800/3805/4882: loss 2.5346, time 66.07ms
# iter 3900/3905/4882: loss 2.5386, time 73.50ms
# step 4000: train loss 2.5350, val loss 2.5345
# saving checkpoint to /Users/rodo/src/rgi3/models/connect4-e2e/v1
# iter 4000/4005/4882: loss 2.5424, time 1217.41ms
# iter 4100/4105/4882: loss 2.5290, time 101.01ms
# iter 4200/4205/4882: loss 2.5323, time 61.94ms
# iter 4300/4305/4882: loss 2.5250, time 72.57ms
# iter 4400/4405/4882: loss 2.5243, time 68.38ms
# iter 4500/4505/4882: loss 2.5331, time 73.33ms
# iter 4600/4605/4882: loss 2.5246, time 101.00ms
# iter 4700/4705/4882: loss 2.5336, time 67.27ms
# iter 4800/4805/4882: loss 2.5170, time 79.40ms

# Sanity check models


In [ ]:
# Play single game
result = await play_generation_async(current_model, num_games=1, simulations=NUM_SIMULATIONS)

In [ ]:
# Inspect training data
td_array = [TrajectoryDataset(DATA_DIR, f"gen-{generation_id}", block_size=n_max_context) for generation_id in range(1, NUM_GENERATIONS+1)]

In [ ]:
# [td for td in td_array]
unrolled = [(generation+1, d) for generation, td in enumerate(td_array) for d in td]

# gen, d = unrolled[0], 
# d.action[:2]
# d.value[0]

dd = defaultdict(lambda: defaultdict(lambda: torch.tensor([0., 0.])))

for gen, d in unrolled:
    for g in ['*', gen]:    
        # dd[tuple(tuple(d.action[:0].tolist()))][g] += d.value[0]
        dd[tuple(tuple(d.action[:1].tolist()))][g] += d.value[0]
        dd[tuple(tuple(d.action[:2].tolist()))][g] += d.value[0]
        dd[tuple(tuple(d.action[:3].tolist()))][g] += d.value[0]
        dd[tuple(tuple(d.action[:4].tolist()))][g] += d.value[0]
        dd[tuple(tuple(d.action[:5].tolist()))][g] += d.value[0]
        dd[tuple(tuple(d.action[:6].tolist()))][g] += d.value[0]
        dd[tuple(tuple(d.action[:7].tolist()))][g] += d.value[0]
        dd[tuple(tuple(d.action[:8].tolist()))][g] += d.value[0]
        dd[tuple(tuple(d.action[:9].tolist()))][g] += d.value[0]
        dd[tuple(tuple(d.action[:10].tolist()))][g] += d.value[0]

print(f"len(dd) = {len(dd)}")


In [ ]:
def eval_prefix(model, game, prefix):
    serial_evaluator = ActionHistoryTransformerEvaluator(model, device=device, block_size=block_size, vocab=action_vocab)
    state = game.initial_state()
    for action in prefix:
        state = game.next_state(state, action)
    legal_actions = game.legal_actions(state)
    result = serial_evaluator.evaluate(game, state, legal_actions)
    return result


In [ ]:
## Someting is borked? Player1 win percent should be much higher??
def compare_model_vs_data(model, game, dd):    
    list(dd.items())[10][1]['*'].sum() > 100
    top_k = sorted(dd.items(), key=lambda kv: kv[1]['*'].sum(), reverse=True)[:20]
    top_k_keys = sorted(k for k, v in top_k)
    
    prefix_list = top_k_keys

    # prefix_list = [
    #     (0,), 
    #     (0,1), (0,2), (0,3), (0,4), (0,5), (0,6), (0,7),
    #     (0,1,1), (0,1,2), (0,1,3), (0,1,4), (0,1,5), (0,1,6), (0,1,7),
    #     (0,4,1), (0,4,2), (0,4,3), (0,4,4), (0,4,5), (0,4,6), (0,4,7),
    # ]

    for prefix in prefix_list:
        print(f"\nprefix={prefix}")
        for gen, counts in dd[prefix].items():
            print(f"gen={gen}: {counts}, win_pct={100*counts[0]/sum(counts):.2f}%, sum={sum(counts)}")
        assert prefix[0] == 0
        actions = prefix[1:]
        eval_result = eval_prefix(model, game, actions)
        # print(f'legal_policy={eval_result.legal_policy}')
        # print(f'player_values={eval_result.player_values}')
        print(f'player_probs={(eval_result.player_values+1)/2}')

compare_model_vs_data(current_model, game, dd)


In [ ]:
# Copy model
model_0 = create_random_model(model_config, action_vocab_size=action_vocab.vocab_size, num_players=game.num_players(state_0), seed=42, device=device)
if RUN_GENERATIONS:
    model_1 = load_model(1)


In [ ]:
print("\n\n### Model 0")
print(model_0.action_embedding.weight)
compare_model_vs_data(model_0, game, dd)

In [ ]:
if RUN_GENERATIONS:
    print("\n\n### Model 1")
    print(model_1.action_embedding.weight)
    compare_model_vs_data(model_1, game, dd)

## Run tournament to calcualte ELO


In [ ]:
import asyncio
import numpy as np
from contextlib import asynccontextmanager
from rgi.rgizero.tournament import Tournament
from rgi.rgizero.players.alphazero import AlphazeroPlayer
from rgi.rgizero.models.action_history_transformer import ActionHistoryTransformerEvaluator, AsyncNetworkEvaluator

@asynccontextmanager
async def create_player_factory(model, simulations, game, device, block_size, action_vocab, max_batch_size):
    """
    Creates a shared evaluator and returns a factory function that produces 
    new AlphazeroPlayer instances using that shared evaluator.
    """
    # 1. Setup the shared evaluator
    serial_evaluator = ActionHistoryTransformerEvaluator(
        model, 
        device=device, 
        block_size=block_size, 
        vocab=action_vocab
    )
    async_evaluator = AsyncNetworkEvaluator(
        base_evaluator=serial_evaluator, 
        max_batch_size=max_batch_size, 
        verbose=False
    )
    
    # 2. Start the evaluator background task
    await async_evaluator.start()
    
    try:
        # 3. Define the factory. This is called by Tournament for every game.
        # It creates a NEW player instance but uses the SHARED async_evaluator.
        def player_factory():
            # Create a fresh RNG for each game/player instance
            rng = np.random.default_rng(np.random.randint(0, 2**31))
            return AlphazeroPlayer(
                game, 
                async_evaluator, 
                rng=rng, 
                add_noise=True, 
                simulations=simulations
            )
            
        yield player_factory
        
    finally:
        # 4. Cleanup
        await async_evaluator.stop()

async def run_tournament_async():
    # Use async with to manage the lifecycle of the evaluators
    async with (
        # create_player_factory(model_dict[0], 100, game, device, block_size, action_vocab, 10) as factory_gen0_100,
        # create_player_factory(model_dict[1], 100, game, device, block_size, action_vocab, 10) as factory_gen1_100,
        # create_player_factory(model_dict[2], 100, game, device, block_size, action_vocab, 10) as factory_gen2_100,
        # create_player_factory(model_dict[3], 100, game, device, block_size, action_vocab, 10) as factory_gen3_100,
        # create_player_factory(model_dict[4], 100, game, device, block_size, action_vocab, 10) as factory_gen4_100,
        # create_player_factory(model_dict[5], 100, game, device, block_size, action_vocab, 10) as factory_gen5_100,
        # create_player_factory(model_dict[10], 100, game, device, block_size, action_vocab, 10) as factory_gen6_100,
        # create_player_factory(model_dict[15], 100, game, device, block_size, action_vocab, 10) as factory_gen7_100,
        # create_player_factory(model_dict[20], 100, game, device, block_size, action_vocab, 10) as factory_gen8_100,

        create_player_factory(model_dict[0], 200, game, device, block_size, action_vocab, 10) as factory_gen0_200,
        create_player_factory(model_dict[1], 200, game, device, block_size, action_vocab, 10) as factory_gen1_200,
        create_player_factory(model_dict[2], 200, game, device, block_size, action_vocab, 10) as factory_gen2_200,
        create_player_factory(model_dict[3], 200, game, device, block_size, action_vocab, 10) as factory_gen3_200,
        create_player_factory(model_dict[4], 200, game, device, block_size, action_vocab, 10) as factory_gen4_200,
        create_player_factory(model_dict[5], 200, game, device, block_size, action_vocab, 10) as factory_gen5_200,
        create_player_factory(model_dict[10], 200, game, device, block_size, action_vocab, 10) as factory_gen10_200,
        create_player_factory(model_dict[15], 200, game, device, block_size, action_vocab, 10) as factory_gen15_200,
        create_player_factory(model_dict[20], 200, game, device, block_size, action_vocab, 10) as factory_gen20_200,
        ):
        
        # The dictionary now maps names to FACTORIES (Callables), not Player instances
        player_factories = {
            # "factory_gen0_100": factory_gen0_100,
            # "factory_gen1_100": factory_gen1_100,
            # "factory_gen2_100": factory_gen2_100,
            # "factory_gen3_100": factory_gen3_100,
            # "factory_gen4_100": factory_gen4_100,
            # "factory_gen5_100": factory_gen5_100,
            # "factory_gen6_100": factory_gen6_100,
            # "factory_gen7_100": factory_gen7_100,

            "factory_gen0_200": factory_gen0_200,
            "factory_gen1_200": factory_gen1_200,
            "factory_gen2_200": factory_gen2_200,
            #"factory_gen3_200": factory_gen3_200,
            #"factory_gen4_200": factory_gen4_200,
            "factory_gen5_200": factory_gen5_200,
            "factory_gen10_200": factory_gen10_200,
            #"factory_gen15_200": factory_gen15_200,
            "factory_gen20_200": factory_gen20_200,
        }
        
        tournament = Tournament(game, player_factories, initial_elo=1000)
        
        print("Running tournament...")
        await tournament.run(num_games=1_000, concurrent_games=2000)
        tournament.print_standings()

if RUN_TOURNAMENT:
    await run_tournament_async()

# Running tournament...
# Tournament Progress: 100%|██████████| 10000/10000 [1:25:59<00:00,  1.94it/s]

# Tournament Standings:
# Rank  Player               ELO        Games    W-L-D          
# -----------------------------------------------------------------
# 1     factory_gen6_200     1140.5     1247     827-419-1      
# 2     factory_gen2_200     1100.1     1251     693-554-4      
# 3     factory_gen5_100     1074.4     1251     598-652-1      
# 4     factory_gen3_200     1029.1     1252     674-573-5      
# 5     factory_gen4_200     1027.0     1248     711-536-1      
# 6     factory_gen0_200     1020.0     1254     444-810-0      
# 7     factory_gen5_200     990.2      1248     742-502-4      
# 8     factory_gen7_100     987.5      1250     650-597-3      
# 9     factory_gen7_200     979.2      1248     768-476-4      
# 10    factory_gen2_100     974.0      1249     522-723-4      
# 11    factory_gen6_100     966.6      1248     684-564-0      
# 12    factory_gen4_100     964.2      1251     557-693-1      
# 13    factory_gen1_100     962.5      1252     547-705-0      
# 14    factory_gen3_100     947.0      1251     528-723-0      
# 15    factory_gen1_200     941.1      1252     630-620-2      
# 16    factory_gen0_100     896.5      1248     410-838-0     


## 20 generations.
# Running tournament...
# Tournament Progress: 100%|██████████| 1000/1000 [08:35<00:00,  1.94it/s]

# Tournament Standings:
# Rank  Player               ELO        Games    W-L-D          
# -----------------------------------------------------------------
# 1     factory_gen10_200    1114.2     333      212-120-1      
# 2     factory_gen2_200     1032.6     333      190-141-2      
# 3     factory_gen1_200     1003.9     334      159-175-0      
# 4     factory_gen20_200    1000.9     335      171-164-0      
# 5     factory_gen5_200     974.6      331      183-146-2      
# 6     factory_gen0_200     873.8      334      82-251-1  

# Tune Model


In [ ]:
from rgi.rgizero.models.tuner import clear_failures_from_cache_file
clear_failures_from_cache_file('result_cache-v0.0.2.json')

In [ ]:
reload_local_modules()

In [ ]:
reload_local_modules(verbose=False)

# Parameters which will never be used for tuning.
fixed_params = dict(
    model_name='c4-tuning',
    model_version='0.1',
    num_players = game.num_players(state_0),
    vocab_size = action_vocab.vocab_size,
    num_genrations = NUM_GENERATIONS,
    data_dir = DATA_DIR,

    eval_iters = 200,
    log_interval = 1000,
    eval_interval = 10_000,

    device = device,
)

initial_params = dict(
    n_layer=2,
    n_head=2,
    n_embd=8,  # tiny model

    n_max_context=n_max_context,
    batch_size = 32,
    gradient_accumulation_steps = 1,

    max_iters=100,
    max_epochs=1_000_000, # Make max_epoch high, rely on max_iters to stop.
        
    learning_rate = LEARNING_RATE,    
    decay_lr = True,  # whether to decay the learning rate
    lr_decay_iters = 100,  # make equal to max_iters usually
    min_lr = LEARNING_RATE / 10,  # learning_rate / 10 usually
    warmup_iters = 0,  # not super necessary potentially

    weight_decay = 1e-1,
    beta1 = 0.9,
    beta2 = 0.95,
    grad_clip = 1.0,  # clip gradients at this value, or disable if == 0.0

    dtype = "float16",

    dropout = 0.0,
    bias = False,  # True: bias in Linears and LayerNorms, like GPT-2. False: a bit better and faster
)

tune_options = dict(
    n_layer = [1, 2, 3, 4, 5, 6, 8, 10, 12, 16, 32],
    # n_head = [1, 2, 4, 8, 16, 32],   # Needs to be calcualted to ensure n_embed % n_head == 0
    n_embd = [8, 16, 32, 64, 128, 256, 512, 1024, 2048],

    n_max_context = [initial_params['n_max_context']],
    batch_size = [16, 32, 64, 128, 256, 512, 1024],
    gradient_accumulation_steps = [1],  # TODO: We only support 1 for now. This fails is we don't have an exact multiple of the batch size per epoch.

    max_iters = [100, 300, 1_000, 3_000, 5_000, 10_000, 30_000, 100_000, 300_000],
    max_epochs = [1_000_000], # Make max_epoch high, rely on max_iters to stop.
 
    learning_rate = [0.001, 0.002, 0.005, 0.01, 0.002, 0.05, 0.1, 0.2, 0.5, 1.0],
    decay_lr = [False, True],

    # TODO: What is a sensible range here?
    beta1 = [0.90, 0.95, 0.99],
    beta2 = [0.95, 0.98, 0.99],

    weight_decay = [0.01, 0.05, 0.1, 0.2],
    grad_clip = [0,0, 1.0],  # clip gradients at this value, or disable if == 0.0

    dtype = ["bfloat16", "float16"],
    dropout = [0.0, 0.01, 0.05, 0.1],
    bias = [True, False],    
)

_n_head_options = [1, 2, 4, 8, 16, 32]
computed_tune_options = dict(
    min_lr = lambda opt: [opt['learning_rate'] / 10],
    lr_decay_iters = lambda opt: [opt['max_iters']],
    warmup_iters = lambda opt: [x for x in [0, 100, 500, 1000] if x < opt['lr_decay_iters']] if opt['decay_lr'] else [0],
    n_head = lambda opt: [n for n in _n_head_options if opt['n_embd'] % n == 0],
)

TUNER_VERSION = "0.0.3"

from rgi.rgizero.models.tuner import Tuner

tuner = Tuner(
    fixed_params=fixed_params.copy(),
    initial_params=initial_params.copy(),
    tune_options=tune_options.copy(), 
    computed_tune_options=computed_tune_options.copy(),
    cache_version=TUNER_VERSION,
    target_improvement_per_minute=1.00)
tuner.autotune_smart()


transform_config_fields: {'bias', 'n_embd', 'n_max_context', 'dropout', 'n_layer', 'n_head'}
train_config_fields: {'batch_size', 'min_lr', 'log_interval', 'eval_iters', 'learning_rate', 'lr_decay_iters', 'max_iters', 'eval_interval', 'model_version', 'grad_clip', 'weight_decay', 'compile', 'always_save_checkpoint', 'eval_only', 'warmup_iters', 'wandb_log', 'gradient_accumulation_steps', 'decay_lr', 'max_epochs', 'device', 'beta2', 'model_name', 'beta1', 'dtype'}
Using initial model as baseline.
## Initial Model, loss=2.4571030139923096 elapsed=6.595702886581421s
## Searching generation 0 with 16 candidates, including ['dropout: 0.01 -> 0.0', 'beta2: 0.95 -> 0.99', 'decay_lr: True -> False', 'learning_rate: 0.05 -> 0.1', 'bias: False -> True']
## Model dropout: 0.01 -> 0.0, loss=2.4571030139923096 elapsed=6.595702886581421s
## Model beta2: 0.95 -> 0.99, loss=2.4571030139923096 elapsed=6.595702886581421s
## Model decay_lr: True -> False, loss=2.4571030139923096 elapsed=6.595702886581421s

(False,
 2.4571030139923096,
 6.595702886581421,
 {'batch_size': 32,
  'beta1': 0.9,
  'beta2': 0.95,
  'bias': False,
  'decay_lr': True,
  'dropout': 0.01,
  'dtype': 'bfloat16',
  'grad_clip': 0,
  'gradient_accumulation_steps': 1,
  'learning_rate': 0.05,
  'lr_decay_iters': 100,
  'max_epochs': 1000000,
  'max_iters': 100,
  'min_lr': 0.005,
  'n_embd': 8,
  'n_head': 2,
  'n_layer': 2,
  'n_max_context': 44,
  'warmup_iters': 0,
  'weight_decay': 0.01,
  'model_name': 'c4-tuning',
  'model_version': '0.1',
  'num_players': 2,
  'vocab_size': 8,
  'num_genrations': 20,
  'data_dir': PosixPath('/Users/rodo/src/rgi3/data/rgizero-e2e/Connect4Game/trajectory_sims-200_games-10000_size-tiny_train-488_x1'),
  'eval_iters': 200,
  'log_interval': 1000,
  'eval_interval': 10000,
  'device': 'mps'})

In [58]:
tuner = Tuner(
    fixed_params=fixed_params.copy(),
    initial_params=initial_params.copy(),
    tune_options=tune_options.copy(), 
    computed_tune_options=computed_tune_options.copy(),
    cache_version=TUNER_VERSION,
    target_improvement_per_minute=0.1)
tuner.autotune_smart()


Using initial model as baseline.
## Initial Model, loss=2.1770522594451904 elapsed=74.7136869430542s
## Searching generation 0 with 18 candidates, including ['max_iters: 3000 -> 1000', 'learning_rate: 0.01 -> 0.05', 'decay_lr: True -> False', 'warmup_iters: 1000 -> 100', 'bias: False -> True']
## Model max_iters: 3000 -> 1000, loss=2.1770522594451904 elapsed=74.7136869430542s
## Model learning_rate: 0.01 -> 0.05, loss=2.1770522594451904 elapsed=74.7136869430542s
## Model decay_lr: True -> False, loss=2.1770522594451904 elapsed=74.7136869430542s
## Model warmup_iters: 1000 -> 100, loss=2.1770522594451904 elapsed=74.7136869430542s
## Model bias: False -> True, loss=2.1770522594451904 elapsed=74.7136869430542s
## Model weight_decay: 0.1 -> 0.01, loss=2.1770522594451904 elapsed=74.7136869430542s
## Model learning_rate: 0.01 -> 0.005, loss=2.1770522594451904 elapsed=74.7136869430542s
## Model n_head: 8 -> 4, loss=2.1770522594451904 elapsed=74.7136869430542s
## Model dtype: bfloat16 -> float

(False,
 2.1770522594451904,
 74.7136869430542,
 {'batch_size': 256,
  'beta1': 0.9,
  'beta2': 0.99,
  'bias': False,
  'decay_lr': True,
  'dropout': 0.0,
  'dtype': 'bfloat16',
  'grad_clip': 0,
  'gradient_accumulation_steps': 1,
  'learning_rate': 0.01,
  'lr_decay_iters': 3000,
  'max_epochs': 1000000,
  'max_iters': 3000,
  'min_lr': 0.001,
  'n_embd': 64,
  'n_head': 8,
  'n_layer': 2,
  'n_max_context': 44,
  'warmup_iters': 1000,
  'weight_decay': 0.1,
  'model_name': 'c4-tuning',
  'model_version': '0.1',
  'num_players': 2,
  'vocab_size': 8,
  'num_genrations': 20,
  'data_dir': PosixPath('/Users/rodo/src/rgi3/data/rgizero-e2e/Connect4Game/trajectory_sims-200_games-10000_size-tiny_train-488_x1'),
  'eval_iters': 200,
  'log_interval': 1000,
  'eval_interval': 10000,
  'device': 'mps'})

In [59]:
tuner = Tuner(
    fixed_params=fixed_params.copy(),
    initial_params=initial_params.copy(),
    tune_options=tune_options.copy(), 
    computed_tune_options=computed_tune_options.copy(),
    cache_version=TUNER_VERSION,
    target_improvement_per_minute=0.01)
tuner.autotune_smart()


Using initial model as baseline.
## Initial Model, loss=2.1298508644104004 elapsed=171.78943705558777s
## Searching generation 0 with 19 candidates, including ['bias: False -> True', 'dtype: bfloat16 -> float16', 'learning_rate: 0.005 -> 0.001', 'n_layer: 3 -> 4', 'n_head: 4 -> 8']
## Model bias: False -> True, loss=2.1298508644104004 elapsed=171.78943705558777s
## Model dtype: bfloat16 -> float16, loss=2.1298508644104004 elapsed=171.78943705558777s
## Model learning_rate: 0.005 -> 0.001, loss=2.1298508644104004 elapsed=171.78943705558777s
## Model n_layer: 3 -> 4, loss=2.1298508644104004 elapsed=171.78943705558777s
## Model n_head: 4 -> 8, loss=2.1298508644104004 elapsed=171.78943705558777s
## Model n_embd: 128 -> 64, loss=2.1298508644104004 elapsed=171.78943705558777s
## Model batch_size: 512 -> 256, loss=2.1298508644104004 elapsed=171.78943705558777s
## Model dropout: 0.0 -> 0.01, loss=2.1298508644104004 elapsed=171.78943705558777s
## Model batch_size: 512 -> 1024, loss=2.1298508644

(False,
 2.1298508644104004,
 171.78943705558777,
 {'batch_size': 512,
  'beta1': 0.9,
  'beta2': 0.99,
  'bias': False,
  'decay_lr': True,
  'dropout': 0.0,
  'dtype': 'bfloat16',
  'grad_clip': 0,
  'gradient_accumulation_steps': 1,
  'learning_rate': 0.005,
  'lr_decay_iters': 3000,
  'max_epochs': 1000000,
  'max_iters': 3000,
  'min_lr': 0.0005,
  'n_embd': 128,
  'n_head': 4,
  'n_layer': 3,
  'n_max_context': 44,
  'warmup_iters': 1000,
  'weight_decay': 0.1,
  'model_name': 'c4-tuning',
  'model_version': '0.1',
  'num_players': 2,
  'vocab_size': 8,
  'num_genrations': 20,
  'data_dir': PosixPath('/Users/rodo/src/rgi3/data/rgizero-e2e/Connect4Game/trajectory_sims-200_games-10000_size-tiny_train-488_x1'),
  'eval_iters': 200,
  'log_interval': 1000,
  'eval_interval': 10000,
  'device': 'mps'})

In [60]:
reload_local_modules(verbose=False)

tuner = Tuner(
    fixed_params=fixed_params.copy(),
    initial_params=initial_params.copy(),
    tune_options=tune_options.copy(), 
    computed_tune_options=computed_tune_options.copy(),
    cache_version=TUNER_VERSION,
    target_improvement_per_minute=0.001)
tuner.autotune_smart()


transform_config_fields: {'bias', 'n_embd', 'n_max_context', 'dropout', 'n_layer', 'n_head'}
train_config_fields: {'batch_size', 'min_lr', 'log_interval', 'eval_iters', 'learning_rate', 'lr_decay_iters', 'max_iters', 'eval_interval', 'model_version', 'grad_clip', 'weight_decay', 'compile', 'always_save_checkpoint', 'eval_only', 'warmup_iters', 'wandb_log', 'gradient_accumulation_steps', 'decay_lr', 'max_epochs', 'device', 'beta2', 'model_name', 'beta1', 'dtype'}
Using initial model as baseline.
## Initial Model, loss=2.119516372680664 elapsed=419.40264201164246s
## Searching generation 0 with 19 candidates, including ['learning_rate: 0.01 -> 0.005', 'bias: False -> True', 'dtype: bfloat16 -> float16', 'beta2: 0.95 -> 0.99', 'n_layer: 3 -> 4']
## Model learning_rate: 0.01 -> 0.005, loss=2.119516372680664 elapsed=419.40264201164246s
## Model bias: False -> True, loss=2.119516372680664 elapsed=419.40264201164246s
## Model dtype: bfloat16 -> float16, loss=2.119516372680664 elapsed=419.4026

(False,
 2.119516372680664,
 419.40264201164246,
 {'batch_size': 1024,
  'beta1': 0.9,
  'beta2': 0.95,
  'bias': False,
  'decay_lr': True,
  'dropout': 0.01,
  'dtype': 'bfloat16',
  'grad_clip': 0,
  'gradient_accumulation_steps': 1,
  'learning_rate': 0.01,
  'lr_decay_iters': 3000,
  'max_epochs': 1000000,
  'max_iters': 3000,
  'min_lr': 0.001,
  'n_embd': 128,
  'n_head': 4,
  'n_layer': 3,
  'n_max_context': 44,
  'warmup_iters': 1000,
  'weight_decay': 0.1,
  'model_name': 'c4-tuning',
  'model_version': '0.1',
  'num_players': 2,
  'vocab_size': 8,
  'num_genrations': 20,
  'data_dir': PosixPath('/Users/rodo/src/rgi3/data/rgizero-e2e/Connect4Game/trajectory_sims-200_games-10000_size-tiny_train-488_x1'),
  'eval_iters': 200,
  'log_interval': 1000,
  'eval_interval': 10000,
  'device': 'mps'})

In [61]:
tuner = Tuner(
    fixed_params=fixed_params.copy(),
    initial_params=initial_params.copy(),
    tune_options=tune_options.copy(), 
    computed_tune_options=computed_tune_options.copy(),
    cache_version=TUNER_VERSION,
    target_improvement_per_minute=0.0001)
tuner.autotune_smart()


Using initial model as baseline.
## Initial Model, loss=2.1182596683502197 elapsed=665.5098741054535s
## Searching generation 0 with 18 candidates, including ['n_layer: 2 -> 3', 'learning_rate: 0.01 -> 0.005', 'dtype: bfloat16 -> float16', 'bias: False -> True', 'beta2: 0.95 -> 0.99']
## Model n_layer: 2 -> 3, loss=2.1182596683502197 elapsed=665.5098741054535s
## Model learning_rate: 0.01 -> 0.005, loss=2.1182596683502197 elapsed=665.5098741054535s
## Model dtype: bfloat16 -> float16, loss=2.1182596683502197 elapsed=665.5098741054535s
## Model bias: False -> True, loss=2.1182596683502197 elapsed=665.5098741054535s
## Model beta2: 0.95 -> 0.99, loss=2.1182596683502197 elapsed=665.5098741054535s
## Model dropout: 0.01 -> 0.05, loss=2.1182596683502197 elapsed=665.5098741054535s
## Model dropout: 0.01 -> 0.0, loss=2.1182596683502197 elapsed=665.5098741054535s
## Model n_head: 8 -> 16, loss=2.1182596683502197 elapsed=665.5098741054535s
## Model max_iters: 3000 -> 10000, loss=2.1182596683502

(False,
 2.1182596683502197,
 665.5098741054535,
 {'batch_size': 1024,
  'beta1': 0.9,
  'beta2': 0.95,
  'bias': False,
  'decay_lr': True,
  'dropout': 0.01,
  'dtype': 'bfloat16',
  'grad_clip': 0,
  'gradient_accumulation_steps': 1,
  'learning_rate': 0.01,
  'lr_decay_iters': 3000,
  'max_epochs': 1000000,
  'max_iters': 3000,
  'min_lr': 0.001,
  'n_embd': 256,
  'n_head': 8,
  'n_layer': 2,
  'n_max_context': 44,
  'warmup_iters': 1000,
  'weight_decay': 0.1,
  'model_name': 'c4-tuning',
  'model_version': '0.1',
  'num_players': 2,
  'vocab_size': 8,
  'num_genrations': 20,
  'data_dir': PosixPath('/Users/rodo/src/rgi3/data/rgizero-e2e/Connect4Game/trajectory_sims-200_games-10000_size-tiny_train-488_x1'),
  'eval_iters': 200,
  'log_interval': 1000,
  'eval_interval': 10000,
  'device': 'mps'})

In [71]:
reload_local_modules(verbose=False)

tuner = Tuner(
    fixed_params=fixed_params.copy(),
    tune_options=tune_options.copy(), 
    initial_params=initial_params.copy(),
    computed_tune_options=computed_tune_options.copy(),
    cache_version=TUNER_VERSION,
    target_improvement_per_minute=0.0)
tuner.autotune_smart()

transform_config_fields: {'bias', 'n_embd', 'n_max_context', 'dropout', 'n_layer', 'n_head'}
train_config_fields: {'batch_size', 'min_lr', 'log_interval', 'eval_iters', 'learning_rate', 'lr_decay_iters', 'max_iters', 'eval_interval', 'model_version', 'grad_clip', 'weight_decay', 'compile', 'always_save_checkpoint', 'eval_only', 'warmup_iters', 'wandb_log', 'gradient_accumulation_steps', 'decay_lr', 'max_epochs', 'device', 'beta2', 'model_name', 'beta1', 'dtype'}
Using initial model as baseline.
## Initial Model, loss=2.1182596683502197 elapsed=665.5098741054535s
## Searching generation 0 with 18 candidates, including ['n_layer: 2 -> 3', 'learning_rate: 0.01 -> 0.005', 'dtype: bfloat16 -> float16', 'bias: False -> True', 'beta2: 0.95 -> 0.99']
## improved: False, loss=2.1217 elapsed=962.37s, mutation n_layer: 2 -> 3
## improved: False, loss=2.1277 elapsed=667.76s, mutation learning_rate: 0.01 -> 0.005
## improved: False, loss=2.1197 elapsed=666.63s, mutation dtype: bfloat16 -> float16
#

(False,
 2.1182596683502197,
 665.5098741054535,
 {'batch_size': 1024,
  'beta1': 0.9,
  'beta2': 0.95,
  'bias': False,
  'decay_lr': True,
  'dropout': 0.01,
  'dtype': 'bfloat16',
  'grad_clip': 0,
  'gradient_accumulation_steps': 1,
  'learning_rate': 0.01,
  'lr_decay_iters': 3000,
  'max_epochs': 1000000,
  'max_iters': 3000,
  'min_lr': 0.001,
  'n_embd': 256,
  'n_head': 8,
  'n_layer': 2,
  'n_max_context': 44,
  'warmup_iters': 1000,
  'weight_decay': 0.1,
  'model_name': 'c4-tuning',
  'model_version': '0.1',
  'num_players': 2,
  'vocab_size': 8,
  'num_genrations': 20,
  'data_dir': PosixPath('/Users/rodo/src/rgi3/data/rgizero-e2e/Connect4Game/trajectory_sims-200_games-10000_size-tiny_train-488_x1'),
  'eval_iters': 200,
  'log_interval': 1000,
  'eval_interval': 10000,
  'device': 'mps'})

In [63]:
from pprint import pprint
print(f'tuner.best_loss={tuner.best_loss}')
print(f'tuner.best_loss_elapsed={int(tuner.best_loss_elapsed)//60}m{tuner.best_loss_elapsed%60:.0f}s')
pprint(tuner.best_params)
# best_params = tuner.initial_params


tuner.best_loss=2.1182596683502197
tuner.best_loss_elapsed=11m6s
{'batch_size': 1024,
 'beta1': 0.9,
 'beta2': 0.95,
 'bias': False,
 'data_dir': PosixPath('/Users/rodo/src/rgi3/data/rgizero-e2e/Connect4Game/trajectory_sims-200_games-10000_size-tiny_train-488_x1'),
 'decay_lr': True,
 'device': 'mps',
 'dropout': 0.01,
 'dtype': 'bfloat16',
 'eval_interval': 10000,
 'eval_iters': 200,
 'grad_clip': 0,
 'gradient_accumulation_steps': 1,
 'learning_rate': 0.01,
 'log_interval': 1000,
 'lr_decay_iters': 3000,
 'max_epochs': 1000000,
 'max_iters': 3000,
 'min_lr': 0.001,
 'model_name': 'c4-tuning',
 'model_version': '0.1',
 'n_embd': 256,
 'n_head': 8,
 'n_layer': 2,
 'n_max_context': 44,
 'num_genrations': 20,
 'num_players': 2,
 'vocab_size': 8,
 'warmup_iters': 1000,
 'weight_decay': 0.1}


## Print tuner stats


In [65]:
reload_local_modules(verbose=False)
tuner = Tuner(
    fixed_params=fixed_params.copy(),
    tune_options=tune_options.copy(), 
    initial_params=initial_params.copy(),
    computed_tune_options=computed_tune_options.copy(),
    cache_version=TUNER_VERSION,
    target_improvement_per_minute=0.001)
# print stats based on cached results.
tuner_stats = tuner.print_hparam_stats()

transform_config_fields: {'bias', 'n_embd', 'n_max_context', 'dropout', 'n_layer', 'n_head'}
train_config_fields: {'batch_size', 'min_lr', 'log_interval', 'eval_iters', 'learning_rate', 'lr_decay_iters', 'max_iters', 'eval_interval', 'model_version', 'grad_clip', 'weight_decay', 'compile', 'always_save_checkpoint', 'eval_only', 'warmup_iters', 'wandb_log', 'gradient_accumulation_steps', 'decay_lr', 'max_epochs', 'device', 'beta2', 'model_name', 'beta1', 'dtype'}


In [66]:
tuner_stats

{('batch_size', 32, 64): {'mean_val_1': 2.4262313842773438,
  'mean_val_2': 2.427396059036255,
  'mean_val_delta': -0.0011646747589111328,
  'mean_elapsed_1': 12.593568960825602,
  'mean_elapsed_2': 20.30130648612976,
  'mean_elapsed_delta': -7.707737525304158,
  'std_val_1': 0.045281896346347265,
  'std_val_2': 0.056964638991242635,
  'std_elapsed_1': 6.99471936361113,
  'std_elapsed_2': 6.423647743592639,
  'std_val_delta': -0.00048003449566209975,
  'std_elapsed_delta': -0.6120375843639212},
 ('batch_size', 32, 16): {'mean_val_1': 2.445603677204677,
  'mean_val_2': 2.475351572036743,
  'mean_val_delta': -0.029747894832066128,
  'mean_elapsed_1': 9.79603283745902,
  'mean_elapsed_2': 13.63189162526812,
  'mean_elapsed_delta': -3.8358587878090993,
  'std_val_1': 0.03012953893357168,
  'std_val_2': 0.027850239626323738,
  'std_elapsed_1': 5.139768302462545,
  'std_elapsed_2': 5.434188149702034,
  'std_val_delta': -0.01216382487045814,
  'std_elapsed_delta': -0.3915726755366898},
 ('bat

In [ ]:
# [(k,v['mean_val_delta']) for (k,v) in sorted(tuner_stats.items(), key=lambda x: x[1]['mean_val_delta'], reverse=True)]

for x in  sorted([(v['mean_val_delta'], k, v['mean_val_1'], v['mean_val_2']) for (k,v) in tuner_stats.items() if not np.isnan(v['mean_val_delta'])], reverse=True): print(x)
# sorted([(v['mean_val_delta'], k) for (k,v) in tuner_stats.items() if not np.isnan(v['mean_val_delta'])], reverse=True)


In [ ]:
for x in sorted([(v['std_val_delta'], k, v['mean_val_1'], v['mean_val_2']) for (k,v) in tuner_stats.items() if not np.isnan(v['std_val_delta'])], reverse=True): print(x)


## Debug Convergence

Synthetic sanity-check: train on a toy 2-step game where the first action strongly determines the winner. This verifies the value head and training loop can learn simple patterns.


In [ ]:
xxx STOP HERE xxx

In [ ]:
state_0 = game.initial_state()
all_actions_0 = game.all_actions()

print(all_actions_0)


In [ ]:
import random

def play_random_game_with_fake_reward(game, max_actions) -> dict:
    state = game.initial_state()
    action_history = []
    legal_policies = []
    legal_action_idx_list = []

    all_actions = game.all_actions()
    all_action_idx_map = {action: idx for idx, action in enumerate(all_actions)}

    num_actions = 0
    while not game.is_terminal(state) and num_actions < max_actions:
        current_player = game.current_player_id(state)
        legal_actions = game.legal_actions(state)
        action_idx = random.randrange(len(legal_actions))
        action = legal_actions[action_idx]

        action_history.append(action)
        legal_policies.append(np.ones(len(legal_actions))/len(legal_actions))
        legal_action_idx = np.array([all_action_idx_map[action] for action in legal_actions])
        legal_action_idx_list.append(legal_action_idx)

        state = game.next_state(state, action)
        num_actions += 1

    # Determine outcome
    fake_reward = np.mean(action_history) / len(legal_actions)
    rewards = np.array([fake_reward, 1.0-fake_reward])
    if fake_reward >= 0.5:
        winner = 1
    else:
        winner = 2

    return {
        "winner": winner,
        "rewards": rewards,
        "action_history": action_history,
        "legal_policies": legal_policies,
        "final_state": state,
        "legal_action_idx": legal_action_idx_list,
    }

In [ ]:
play_random_game_with_fake_reward(game, max_actions=2)

In [ ]:
results = [play_random_game_with_fake_reward(game, max_actions=2) for _ in range(100_000)]
print_game_stats(results)


In [ ]:
fake_gen_name = "fake-0"
trajectory_path = write_trajectory_dataset(results, action_vocab, fake_gen_name)


In [ ]:
# fake_model_config = model_config_dict[MODEL_SIZE]
fake_model_config = model_config_dict["large"]
fake_model = create_random_model(fake_model_config, action_vocab_size=action_vocab.vocab_size, num_players=game.num_players(state_0), seed=42, device=device)

training_splits = [f'gen-{fake_gen_name}']
fake_model, fake_trainer = train_model(fake_model, training_splits, train_config)
save_model(fake_model, fake_trainer, fake_gen_name)

## model_size=tiny
# num decayed parameter tensors: 11, with 1,968 parameters
# num non-decayed parameter tensors: 7, with 50 parameters
# using fused AdamW: False
# step 0: train loss 2.7817, val loss 2.7816
# iter 0/49/488: loss 2.7821, time 2537.56ms
# iter 100/147/488: loss 2.6890, time 53.61ms
# iter 200/245/488: loss 2.6342, time 63.05ms
# iter 300/343/488: loss 2.6187, time 55.31ms
# iter 400/441/488: loss 2.6147, time 61.11ms

## model_size=large
# num decayed parameter tensors: 35, with 1,579,776 parameters
# num non-decayed parameter tensors: 19, with 2,186 parameters
# using fused AdamW: False
# step 0: train loss 2.8087, val loss 2.8088
# iter 0/49/488: loss 2.8099, time 11225.20ms
# iter 100/147/488: loss 2.6065, time 596.91ms
# iter 200/245/488: loss 2.6075, time 618.00ms
# iter 300/343/488: loss 2.6080, time 613.63ms
# iter 400/441/488: loss 2.6051, time 616.39ms

In [ ]:
# for rerun in range(10):
#     print(f"Re-running training for {fake_gen_name} {rerun+1} of 10")
#     fake_model, fake_trainer = train_model(fake_model, training_splits, train_config)
#     save_model(fake_model, fake_trainer, fake_gen_name)

In [ ]:
# [td for td in td_array]
fake_td_array = [TrajectoryDataset(DATA_DIR, split, block_size=n_max_context) for split in training_splits]
fake_unrolled = [(generation+1, d) for generation, td in enumerate(fake_td_array) for d in td]

# gen, d = unrolled[0], 
# d.action[:2]
# d.value[0]

# Inspect training data
fake_dd = defaultdict(lambda: defaultdict(lambda: torch.tensor([0., 0.])))

for gen, d in fake_unrolled:
    for g in ['*', gen]:    
        fake_dd[tuple(tuple(d.action[:0].tolist()))][g] += d.value[0]
        fake_dd[tuple(tuple(d.action[:1].tolist()))][g] += d.value[0]
        fake_dd[tuple(tuple(d.action[:2].tolist()))][g] += d.value[0]
        # fake_dd[tuple(tuple(d.action[:3].tolist()))][g] += d.value[0]

print(f"len(fake_dd) = {len(fake_dd)}")


In [ ]:
fake_model = load_model(fake_gen_name)
compare_model_vs_data(fake_model, game, dd)


In [ ]:
fake_model, fake_trainer = train_model(fake_model, training_splits, train_config)
